Function to process the TFRecord and create a csv with the fields - filename, class, xmin, ymin, xmax, ymax, prediction_score

In [2]:
# import modules
import csv, os, sys, io
import tensorflow as tf
import pandas as pd
import numpy as np
sys.path.append('/home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/data/')
from utils import dataset_util
from PIL import ImageFile
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# input parameters
filename_list = ['/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_s01_s06-0-10000.record']
batch_size = 2
score_threshold = 0.5

In [ ]:
c = 0
tfrecords_filename = '/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_s01_s06-0-10000.record'
record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

for s_example in record_iterator:
    example = tf.train.Example()
    example.ParseFromString(s_example)
    print(example)
    
#     box_labels = (example.features.feature["image/object/class/label"]
#                                 .int64_list)
#     box_class = (example.features.feature["image/object/class/text"]
#                                 .bytes_list)
#     example = tf.train.Example.FromString(s_example)
    c +=1
    if c == 1:
#         print(box_labels, box_class)
        print(box_class)
    else :
        break

In [4]:
"""This section contains the helper functions needed for 
decoding TFRecord...
1. decode_record: This function decodes a TFexample with the following features
               'image/filename' - a fixed length feature with the file name
               'image/encoded' - a fixed length feature with image encodings
               'image/format' - a fixed length features with image format
               'image/detection/bbox/xmin' - a varaible name feature with normalized xmin values 
               'image/detection/bbox/xmax' - normalized xmax values
               'image/detection/bbox/ymin' - normalized ymin values
               'image/detection/bbox/ymax' - normalized ymax values
               'image/detection/label' - bounding box labels
               'image/detection/score' - prediction score

"""

def decode_record(serialized_example):
    context_features = {
                        'image/filename': tf.FixedLenFeature([], tf.string),
                        'image/encoded': tf.FixedLenFeature([], tf.string),
                        'image/format': tf.FixedLenFeature([], tf.string),
                        "image/detection/bbox/xmin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/xmax" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymax" : tf.VarLenFeature(tf.float32),
                        "image/detection/label" : tf.VarLenFeature(tf.int64),
                        "image/detection/score" : tf.VarLenFeature(tf.float32)
                    }


    context, sequence = tf.parse_single_sequence_example(serialized=serialized_example,
                                              context_features=context_features,
#                                               sequence_features=sequence_features,
                                              example_name=None,
                                              name=None)

    return ({k: v for k, v in context.items()},{k: v for k, v in sequence.items()})

# Create an iterator to traverse the file
dataset = tf.data.Dataset.from_tensor_slices(filename_list)
dataset = tf.data.TFRecordDataset(dataset)
dataset = dataset.map(lambda x: decode_record(serialized_example=x)).batch(batch_size)

iterator = dataset.make_one_shot_iterator() # create the iterator
batch_data = iterator.get_next()

# Run the session and extract the feature values
with tf.Session() as sess:
    (context, sequence) = sess.run(batch_data)
    img = context['image/encoded']
    filename = context['image/filename']
    # Features added during the detection phase 
    xmin_d = (context['image/detection/bbox/xmin'])
    ymin_d = (context['image/detection/bbox/ymin'])
    xmax_d = (context['image/detection/bbox/xmax'])
    ymax_d = (context['image/detection/bbox/ymax'])
    label_d = (context['image/detection/label'])
    score = (context['image/detection/score'])
    
    file_name = [] # list of mages with no groundtruth
    no_prediction = [] # Storing the image name of the correct predictions
    incorrect_prediction = [] # Storing the image name of the incorrect predictions
    
    # index of the boxes with score greater than the threshold
    index_score_gt_threshold = score[1] > score_threshold # can be checked only once. So moved out of the for loop
    xmins_d, ymins_d, xmaxs_d, ymaxs_d, labels_d, scores = [], [], [], [], [], []
    for rec_i in range(len(img)):
        encoded_jpg_io = io.BytesIO(img[rec_i])
        image = Image.open(encoded_jpg_io)
        width, height = image.size
        
        # index of the image
        index_image_rec_i = xmin_d[0][:, 0] == rec_i
        # index of the boxes with score greater than the threshold
        # index_score_gt_threshold = score[1] > score_threshold
        index_img_gt_thres = list(np.where([a and b for a, b in zip(index_image_rec_i, index_score_gt_threshold)])[0])
        
        xmins_d = xmins_d + list(xmin_d[1][index_img_gt_thres])
        ymins_d = ymins_d + list(ymin_d[1][index_img_gt_thres])
        xmaxs_d = xmaxs_d + list(xmax_d[1][index_img_gt_thres])
        ymaxs_d = ymaxs_d + list(ymax_d[1][index_img_gt_thres])
        labels_d = labels_d + list(label_d[1][index_img_gt_thres])
        scores = scores + list(score[1][index_img_gt_thres])
        file_name = file_name + [filename[rec_i].decode('ascii')]*len(index_img_gt_thres)
        
        if len(index_img_gt_thres)==0:
            no_prediction.append(filename[rec_i].decode('ascii'))

In [4]:
# Create pandas dataframe
df_predictions = pd.DataFrame({'labels':labels_d, 
                               'filename':file_name,
                               'score': scores,
                               'xmin': xmins_d,
                               'ymin': ymins_d,
                               'xmax': xmaxs_d,
                               'ymax': ymaxs_d})
df_predictions = df_predictions.append(pd.DataFrame({'filename':no_prediction}))
# write predictions to csv
# df_predictions.to_csv('/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_s01_s06-0-10000.csv')

/home/ubuntu/test_env/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [ ]:
# context['image/detection/bbox/ymin'][0].shape
xmin_test = tf.sparse_tensor_to_dense(
    context['image/detection/bbox/xmin'],
    default_value=0,
    validate_indices=True,
    name=None
)

with tf.Session() as sess:
    print(sess.run(xmin_test))
    print(sess.run(xmin_test[0,5]))
# context['image/detection/bbox/xmin']

In [19]:
"""Predictor Extractor reads the TFRecords with predictions and outputs the csv file with the predictions """
import tensorflow as tf
import pandas as pd

# tf.enable_eager_execution()

def decode_record(serialized_example):
    """This section contains the helper functions needed for 
    decoding TFRecord...
    1. decode_record: This function decodes a TFexample with the following features
                   'image/filename' - a fixed length feature with the file name
                   'image/encoded' - a fixed length feature with image encodings
                   'image/format' - a fixed length features with image format
                   'image/detection/bbox/xmin' - a varaible name feature with normalized xmin values 
                   'image/detection/bbox/xmax' - normalized xmax values
                   'image/detection/bbox/ymin' - normalized ymin values
                   'image/detection/bbox/ymax' - normalized ymax values
                   'image/detection/label' - bounding box labels
                   'image/detection/score' - prediction score

    """


    context_features = {
                        'image/filename': tf.FixedLenFeature([], tf.string),
                        'image/encoded': tf.FixedLenFeature([], tf.string),
                        'image/format': tf.FixedLenFeature([], tf.string),
                        "image/detection/bbox/xmin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/xmax" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymax" : tf.VarLenFeature(tf.float32),
                        "image/detection/label" : tf.VarLenFeature(tf.int64),
                        "image/detection/score" : tf.VarLenFeature(tf.float32)
                    }


    context, sequence = tf.parse_single_sequence_example(serialized=serialized_example,
                                              context_features=context_features,
#                                               sequence_features=sequence_features,
                                              example_name=None,
                                              name=None)

    return ({k: v for k, v in context.items()},{k: v for k, v in sequence.items()})


tfrecord_path_list = ['/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_s01_s06-0-10000.record']
batch_size = 512
score_threshold = 0.5
output_csv = '/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_test.csv'


dataset = tf.data.Dataset.from_tensor_slices(tfrecord_path_list)
dataset = tf.data.TFRecordDataset(dataset)
dataset = dataset.repeat(1)
dataset = dataset.map(lambda x: decode_record(serialized_example=x)).batch(batch_size)

# iterator = dataset.make_one_shot_iterator() # create the iterator
# batch_data = iterator.get_next()

# Run the session and extract the feature values
# with tf.Session() as sess:
    
xmins_d, ymins_d, xmaxs_d, ymaxs_d, labels_d, scores, filenames = [], [], [], [], [], [], []
filenames_without_predictions = []


for i, (context, sequence) in enumerate(dataset):
    batch_shape = context['image/detection/bbox/xmin'].dense_shape
    #context['image/detection/bbox/xmin'][2]

    img = context['image/encoded']
    filename = context['image/filename']
    # Features added during the detection phase 
    xmin_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/xmin'])
    ymin_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/ymin'])
    xmax_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/xmax'])
    ymax_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/ymax'])
    label_d = tf.sparse_tensor_to_dense(context['image/detection/label'])
    score = tf.sparse_tensor_to_dense(context['image/detection/score'])


    for rec_i in range(0, int(batch_shape[0])):
        box_counter = 0
        for box_i in range(0, int(batch_shape[1])):
            if score[rec_i, box_i] < score_threshold:
                continue
            xmins_d.append(xmin_d[rec_i, box_i].numpy())
            ymins_d.append(xmax_d[rec_i, box_i].numpy())
            xmaxs_d.append(ymax_d[rec_i, box_i].numpy())
            ymaxs_d.append(ymax_d[rec_i, box_i].numpy())
            labels_d.append(label_d[rec_i, box_i].numpy())
            scores.append(score[rec_i, box_i].numpy())
            filenames.append(filename[rec_i].numpy().decode('utf-8'))

            box_counter += 1

        if box_counter == 0:
            filenames_without_predictions.append(filename[rec_i].numpy().decode('utf-8'))
    
    print('image: {0} finished'.format(i))

            
 # Create pandas dataframe
df_predictions = pd.DataFrame({'labels':labels_d, 
                               'filename':filenames,
                               'score': scores,
                               'xmin': xmins_d,
                               'ymin': ymins_d,
                               'xmax': xmaxs_d,
                               'ymax': ymaxs_d})

df_predictions = df_predictions.append(pd.DataFrame({'filename':filenames_without_predictions}))
# write predictions to csv
df_predictions.to_csv(output_csv, index=False)


image: 0 finished
image: 1 finished
image: 2 finished
image: 3 finished
image: 4 finished
image: 5 finished
image: 6 finished
image: 7 finished
image: 8 finished
image: 9 finished
image: 10 finished
image: 11 finished
image: 12 finished
image: 13 finished
image: 14 finished
image: 15 finished
image: 16 finished
image: 17 finished
image: 18 finished
image: 19 finished


/home/ubuntu/test_env/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [9]:
%%writefile /home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/data_prep/predictor_extractor.py
"""Predictor Extractor reads the TFRecords with predictions and outputs the csv file with the predictions """
import tensorflow as tf
import pandas as pd

tf.enable_eager_execution()

def decode_record(serialized_example, discard_image_pixels=True):
    """This section contains the helper functions needed for 
    decoding TFRecord...
    1. decode_record: This function decodes a TFexample with the following features
                   'image/filename' - a fixed length feature with the file name
                   'image/encoded' - a fixed length feature with image encodings
                   'image/format' - a fixed length features with image format
                   'image/detection/bbox/xmin' - a varaible name feature with normalized xmin values 
                   'image/detection/bbox/xmax' - normalized xmax values
                   'image/detection/bbox/ymin' - normalized ymin values
                   'image/detection/bbox/ymax' - normalized ymax values
                   'image/detection/label' - bounding box labels
                   'image/detection/score' - prediction score

    """


    context_features = {
                        'image/filename': tf.FixedLenFeature([], tf.string),
                        # 'image/encoded': tf.FixedLenFeature([], tf.string),
                        'image/format': tf.FixedLenFeature([], tf.string),
                        "image/detection/bbox/xmin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/xmax" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymax" : tf.VarLenFeature(tf.float32),
                        "image/detection/label" : tf.VarLenFeature(tf.int64),
                        "image/detection/score" : tf.VarLenFeature(tf.float32)
                    }


    context, sequence = tf.parse_single_sequence_example(serialized=serialized_example,
                                              context_features=context_features,
#                                               sequence_features=sequence_features,
                                              example_name=None,
                                              name=None)

    return ({k: v for k, v in context.items()},{k: v for k, v in sequence.items()})


def predictorExtractor(tfrecord_path_list,
                       output_csv,
                       groundtruth_consolidated_dict,
                       discard_image_pixels=True,
                       batch_size=512, 
                       score_threshold=0.5,
                       is_training='True'):
    """
    This requires a dicrionary with groudtruth file name and total animals in it
    """
    
    dataset = tf.data.Dataset.from_tensor_slices(tfrecord_path_list)
    dataset = tf.data.TFRecordDataset(dataset)
    dataset = dataset.repeat(1)
    dataset = dataset.map(lambda x: decode_record(serialized_example=x)).batch(batch_size)

    xmins_d, ymins_d, xmaxs_d, ymaxs_d, labels_d, scores, filenames = [], [], [], [], [], [], []
    filenames_without_predictions = []


    for i, (context, sequence) in enumerate(dataset):
        batch_shape = context['image/detection/bbox/xmin'].dense_shape
        #context['image/detection/bbox/xmin'][2]
        # if discard_image_pixels!=True:
        # img = context['image/encoded']
        filename = context['image/filename']
        # Features added during the detection phase 
        xmin_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/xmin'])
        ymin_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/ymin'])
        xmax_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/xmax'])
        ymax_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/ymax'])
        label_d = tf.sparse_tensor_to_dense(context['image/detection/label'])
        score = tf.sparse_tensor_to_dense(context['image/detection/score'])


        for rec_i in range(0, int(batch_shape[0])):
            if is_training=='True':
                num_box = groundtruth_consolidated_dict[filename[rec_i].numpy().decode('utf-8')]
                if num_box in ['11-50', '51+']:
                    score_threshold = 0.25
                else:
                    score_threshold = min(score[rec_i, 0:int(num_box)])
            box_counter = 0
            for box_i in range(0, int(batch_shape[1])):
                if score[rec_i, box_i] < score_threshold:
                    continue
                xmins_d.append(xmin_d[rec_i, box_i].numpy())
                ymins_d.append(ymin_d[rec_i, box_i].numpy())
                xmaxs_d.append(xmax_d[rec_i, box_i].numpy())
                ymaxs_d.append(ymax_d[rec_i, box_i].numpy())
                labels_d.append(int(label_d[rec_i, box_i].numpy()))
                scores.append(score[rec_i, box_i].numpy())
                filenames.append(filename[rec_i].numpy().decode('utf-8'))

                box_counter += 1

            if box_counter == 0:
                filenames_without_predictions.append(filename[rec_i].numpy().decode('utf-8'))

        print('Batch: {0} finished'.format(i))
    
     # Create pandas dataframe
    df_predictions = pd.DataFrame({'labels':labels_d, 
                                   'filename':filenames,
                                   'score': scores,
                                   'xmin': xmins_d,
                                   'ymin': ymins_d,
                                   'xmax': xmaxs_d,
                                   'ymax': ymaxs_d})

    df_predictions = df_predictions.append(pd.DataFrame({'filename':filenames_without_predictions}))
    df_predictions = df_predictions.round({'score':2})
    # write predictions to csv
    df_predictions.to_csv(output_csv, index=False)

Overwriting /home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/data_prep/predictor_extractor.py


**main file**

In [8]:
%%writefile /home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/predictorExtractor_main.py
"""Main function for extracting TFRecord to a csv. 
Uses the helper modules predictor_extractor.py"""

import argparse
import pandas as pd

from data_prep.predictor_extractor import predictorExtractor
import data_prep.data_prep_utils as dataprep_utils
import bootstrapping.prediction_groundtruth_consolidation as pgc

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--tfrecord_path_list", nargs='+', type=str, required=True,
        help="Path to TFRecord files")
    parser.add_argument(
        "--output_csv", type=str, required=True,
        help="output csv file"
        )
    parser.add_argument(
        "--batch_size", type=int, default=512,
        help="batch size")
    parser.add_argument(
        "--score_threshold", type=float, default=0.5,
        help="score thresholds to write to csv")
    parser.add_argument(
        "--discard_image_pixels", type=bool, default=True,
        help="True to discard the pixel encodings or when pixel encodings are not present in the datafile")
    parser.add_argument(
        "--groundtruth_csv_path", type=str, required=True,
        help="path to the groundtruth file"
    )
    parser.add_argument(
        "--label_map_json", type=str, required=True,
        help="path to the label map json file")
    parser.add_argument(
        "--is_training", type=str, default='True',
        help="if the data is for the training purposes of bootstrapping step. 1 for training 0 for test")


#     kwargs = vars(parser.parse_args())
    args = parser.parse_args()
    
    label_map_df = pd.DataFrame.from_dict(dataprep_utils.get_label_map_from_json(args.label_map_json), orient='index').reset_index()
    label_map_df.columns=['species', 'labels']

    groundtruth_df_img = pgc.process_grondtruth_classification_data(args.groundtruth_csv_path, label_map_df)
    groundtruth_dict = groundtruth_df_img.to_dict(orient='index')
    groundtruth_consolidated_dict = {}
    
    for k, v in groundtruth_dict.items():
        if v['filename'] not in groundtruth_consolidated_dict.keys():
            groundtruth_consolidated_dict[v['filename']] = v['groundtruth_counts']
        elif v['groundtruth_counts'] == '11-50' and groundtruth_consolidated_dict[v['filename']] != '51+':
            groundtruth_consolidated_dict[v['filename']] = '11-50'
        elif v['groundtruth_counts'] != '51+' and groundtruth_consolidated_dict[v['filename']] == '11-50':
            groundtruth_consolidated_dict[v['filename']] = '11-50'
        elif v['groundtruth_counts'] == '51+' or groundtruth_consolidated_dict[v['filename']] == '51+':
            groundtruth_consolidated_dict[v['filename']] = '51+'
        else:
            groundtruth_consolidated_dict[v['filename']] = int(groundtruth_consolidated_dict[v['filename']])+int(v['groundtruth_counts'])
            
    predictorExtractor(args.tfrecord_path_list, args.output_csv, groundtruth_consolidated_dict, is_training=args.is_training)
#     predictorExtractor(**kwargs)

Overwriting /home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/predictorExtractor_main.py


In [1]:
import sys
sys.path.append('/home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/')
import tensorflow as tf
import pandas as pd
import numpy as np

import bootstrapping.prediction_groundtruth_consolidation as pgc
import data_prep.data_prep_utils as dataprep_utils
# tf.enable_eager_execution()

In [2]:
def decode_record(serialized_example, discard_image_pixels=True):
    """This section contains the helper functions needed for 
    decoding TFRecord...
    1. decode_record: This function decodes a TFexample with the following features
                   'image/filename' - a fixed length feature with the file name
                   'image/encoded' - a fixed length feature with image encodings
                   'image/format' - a fixed length features with image format
                   'image/detection/bbox/xmin' - a varaible name feature with normalized xmin values 
                   'image/detection/bbox/xmax' - normalized xmax values
                   'image/detection/bbox/ymin' - normalized ymin values
                   'image/detection/bbox/ymax' - normalized ymax values
                   'image/detection/label' - bounding box labels
                   'image/detection/score' - prediction score

    """


    context_features = {
                        'image/filename': tf.FixedLenFeature([], tf.string),
                        # 'image/encoded': tf.FixedLenFeature([], tf.string),
                        'image/format': tf.FixedLenFeature([], tf.string),
                        "image/detection/bbox/xmin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/xmax" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymin" : tf.VarLenFeature(tf.float32),
                        "image/detection/bbox/ymax" : tf.VarLenFeature(tf.float32),
                        "image/detection/label" : tf.VarLenFeature(tf.int64),
                        "image/detection/score" : tf.VarLenFeature(tf.float32)
                    }


    context, sequence = tf.parse_single_sequence_example(serialized=serialized_example,
                                              context_features=context_features,
#                                               sequence_features=sequence_features,
                                              example_name=None,
                                              name=None)

    return ({k: v for k, v in context.items()},{k: v for k, v in sequence.items()})



In [45]:
"""Predictor Extractor reads the TFRecords with predictions and outputs the csv file with the predictions """

def predictorExtractor(tfrecord_path_list,
                       output_csv,
                       groundtruth_consolidated_dict,
                       discard_image_pixels=True,
                       batch_size=512, 
                       score_threshold=0.5,
                       is_training=True):
    """
    This requires a dicrionary with groudtruth file name and total animals in it
    """
    
    dataset = tf.data.Dataset.from_tensor_slices(tfrecord_path_list)
    dataset = tf.data.TFRecordDataset(dataset)
    dataset = dataset.repeat(1)
    dataset = dataset.map(lambda x: decode_record(serialized_example=x)).batch(batch_size)

    xmins_d, ymins_d, xmaxs_d, ymaxs_d, labels_d, scores, filenames = [], [], [], [], [], [], []
    filenames_without_predictions = []


    for i, (context, sequence) in enumerate(dataset):
        batch_shape = context['image/detection/bbox/xmin'].dense_shape
        #context['image/detection/bbox/xmin'][2]
        # if discard_image_pixels!=True:
        # img = context['image/encoded']
        filename = context['image/filename']
        # Features added during the detection phase 
        xmin_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/xmin'])
        ymin_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/ymin'])
        xmax_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/xmax'])
        ymax_d = tf.sparse_tensor_to_dense(context['image/detection/bbox/ymax'])
        label_d = tf.sparse_tensor_to_dense(context['image/detection/label'])
        score = tf.sparse_tensor_to_dense(context['image/detection/score'])


        for rec_i in range(0, int(batch_shape[0])):
            if is_training==True:
                num_box = dict1[filename[rec_i].numpy().decode('utf-8')]
                score_threshold = min(score[rec_i, 0:int(num_box)])
            box_counter = 0
            for box_i in range(0, int(batch_shape[1])):
                if score[rec_i, box_i] < score_threshold:
                    continue
                xmins_d.append(xmin_d[rec_i, box_i].numpy())
                ymins_d.append(ymin_d[rec_i, box_i].numpy())
                xmaxs_d.append(xmax_d[rec_i, box_i].numpy())
                ymaxs_d.append(ymax_d[rec_i, box_i].numpy())
                labels_d.append(int(label_d[rec_i, box_i].numpy()))
                scores.append(score[rec_i, box_i].numpy())
                filenames.append(filename[rec_i].numpy().decode('utf-8'))

                box_counter += 1

            if box_counter == 0:
                filenames_without_predictions.append(filename[rec_i].numpy().decode('utf-8'))

        print('Batch: {0} finished'.format(i))
    
     # Create pandas dataframe
    df_predictions = pd.DataFrame({'labels':labels_d, 
                                   'filename':filenames,
                                   'score': scores,
                                   'xmin': xmins_d,
                                   'ymin': ymins_d,
                                   'xmax': xmaxs_d,
                                   'ymax': ymaxs_d})

    df_predictions = df_predictions.append(pd.DataFrame({'filename':filenames_without_predictions}))
    df_predictions = df_predictions.round({'score':2})
    # write predictions to csv
    df_predictions.to_csv(output_csv, index=False)


In [46]:
predictorExtractor(tfrecord_path_list, output_csv, dict1)

1
tf.Tensor([0.9447739], shape=(1,), dtype=float32)
tf.Tensor(0.9447739, shape=(), dtype=float32)
1
tf.Tensor([0.9982875], shape=(1,), dtype=float32)
tf.Tensor(0.9982875, shape=(), dtype=float32)
1
tf.Tensor([0.999006], shape=(1,), dtype=float32)
tf.Tensor(0.999006, shape=(), dtype=float32)
1
tf.Tensor([0.99820614], shape=(1,), dtype=float32)
tf.Tensor(0.99820614, shape=(), dtype=float32)
1
tf.Tensor([0.99184895], shape=(1,), dtype=float32)
tf.Tensor(0.99184895, shape=(), dtype=float32)
1
tf.Tensor([0.9927965], shape=(1,), dtype=float32)
tf.Tensor(0.9927965, shape=(), dtype=float32)
1
tf.Tensor([0.9939014], shape=(1,), dtype=float32)
tf.Tensor(0.9939014, shape=(), dtype=float32)
1
tf.Tensor([0.7897988], shape=(1,), dtype=float32)
tf.Tensor(0.7897988, shape=(), dtype=float32)
1
tf.Tensor([0.9973078], shape=(1,), dtype=float32)
tf.Tensor(0.9973078, shape=(), dtype=float32)
1
tf.Tensor([0.9633884], shape=(1,), dtype=float32)
tf.Tensor(0.9633884, shape=(), dtype=float32)
1
tf.Tensor([0.993

tf.Tensor(0.112652816, shape=(), dtype=float32)
1
tf.Tensor([0.35529417], shape=(1,), dtype=float32)
tf.Tensor(0.35529417, shape=(), dtype=float32)
1
tf.Tensor([0.42889813], shape=(1,), dtype=float32)
tf.Tensor(0.42889813, shape=(), dtype=float32)
1
tf.Tensor([0.96927816], shape=(1,), dtype=float32)
tf.Tensor(0.96927816, shape=(), dtype=float32)
1
tf.Tensor([0.9927327], shape=(1,), dtype=float32)
tf.Tensor(0.9927327, shape=(), dtype=float32)
1
tf.Tensor([0.99350435], shape=(1,), dtype=float32)
tf.Tensor(0.99350435, shape=(), dtype=float32)
1
tf.Tensor([0.99037194], shape=(1,), dtype=float32)
tf.Tensor(0.99037194, shape=(), dtype=float32)
2
tf.Tensor([0.9951605 0.8645659], shape=(2,), dtype=float32)
tf.Tensor(0.8645659, shape=(), dtype=float32)
2
tf.Tensor([0.99431485 0.96936506], shape=(2,), dtype=float32)
tf.Tensor(0.96936506, shape=(), dtype=float32)
2
tf.Tensor([0.99467695 0.9296999 ], shape=(2,), dtype=float32)
tf.Tensor(0.9296999, shape=(), dtype=float32)
4
tf.Tensor([0.9975568  0

tf.Tensor(0.4847101, shape=(), dtype=float32)
2
tf.Tensor([0.6587575  0.60450935], shape=(2,), dtype=float32)
tf.Tensor(0.60450935, shape=(), dtype=float32)
1
tf.Tensor([0.9993703], shape=(1,), dtype=float32)
tf.Tensor(0.9993703, shape=(), dtype=float32)
1
tf.Tensor([0.7653058], shape=(1,), dtype=float32)
tf.Tensor(0.7653058, shape=(), dtype=float32)
2
tf.Tensor([0.765115  0.5890048], shape=(2,), dtype=float32)
tf.Tensor(0.5890048, shape=(), dtype=float32)
1
tf.Tensor([0.57428426], shape=(1,), dtype=float32)
tf.Tensor(0.57428426, shape=(), dtype=float32)
1
tf.Tensor([0.8484249], shape=(1,), dtype=float32)
tf.Tensor(0.8484249, shape=(), dtype=float32)
1
tf.Tensor([0.91206527], shape=(1,), dtype=float32)
tf.Tensor(0.91206527, shape=(), dtype=float32)
1
tf.Tensor([0.9956136], shape=(1,), dtype=float32)
tf.Tensor(0.9956136, shape=(), dtype=float32)
1
tf.Tensor([0.6272254], shape=(1,), dtype=float32)
tf.Tensor(0.6272254, shape=(), dtype=float32)
2
tf.Tensor([0.98879635 0.9814787 ], shape=(2

tf.Tensor([0.99730873 0.8067491 ], shape=(2,), dtype=float32)
tf.Tensor(0.8067491, shape=(), dtype=float32)
2
tf.Tensor([0.99796826 0.9577238 ], shape=(2,), dtype=float32)
tf.Tensor(0.9577238, shape=(), dtype=float32)
2
tf.Tensor([0.9973611 0.9789457], shape=(2,), dtype=float32)
tf.Tensor(0.9789457, shape=(), dtype=float32)
1
tf.Tensor([0.9955669], shape=(1,), dtype=float32)
tf.Tensor(0.9955669, shape=(), dtype=float32)
1
tf.Tensor([0.99819535], shape=(1,), dtype=float32)
tf.Tensor(0.99819535, shape=(), dtype=float32)
1
tf.Tensor([0.8020423], shape=(1,), dtype=float32)
tf.Tensor(0.8020423, shape=(), dtype=float32)
1
tf.Tensor([0.994189], shape=(1,), dtype=float32)
tf.Tensor(0.994189, shape=(), dtype=float32)
1
tf.Tensor([0.994909], shape=(1,), dtype=float32)
tf.Tensor(0.994909, shape=(), dtype=float32)
1
tf.Tensor([0.9938724], shape=(1,), dtype=float32)
tf.Tensor(0.9938724, shape=(), dtype=float32)
2
tf.Tensor([0.9972697 0.9921308], shape=(2,), dtype=float32)
tf.Tensor(0.9921308, shape

tf.Tensor([0.8763225], shape=(1,), dtype=float32)
tf.Tensor(0.8763225, shape=(), dtype=float32)
1
tf.Tensor([0.9778129], shape=(1,), dtype=float32)
tf.Tensor(0.9778129, shape=(), dtype=float32)
1
tf.Tensor([0.9768809], shape=(1,), dtype=float32)
tf.Tensor(0.9768809, shape=(), dtype=float32)
4
tf.Tensor([0.9728487  0.34653255 0.07497538 0.03465631], shape=(4,), dtype=float32)
tf.Tensor(0.034656312, shape=(), dtype=float32)
4
tf.Tensor([0.96145695 0.09308374 0.03642206 0.01177023], shape=(4,), dtype=float32)
tf.Tensor(0.011770225, shape=(), dtype=float32)
4
tf.Tensor([0.95336944 0.4432215  0.10267066 0.07338555], shape=(4,), dtype=float32)
tf.Tensor(0.07338555, shape=(), dtype=float32)
1
tf.Tensor([0.62269205], shape=(1,), dtype=float32)
tf.Tensor(0.62269205, shape=(), dtype=float32)
1
tf.Tensor([0.89054567], shape=(1,), dtype=float32)
tf.Tensor(0.89054567, shape=(), dtype=float32)
1
tf.Tensor([0.91981924], shape=(1,), dtype=float32)
tf.Tensor(0.91981924, shape=(), dtype=float32)
1
tf.Te

1
tf.Tensor([0.8439249], shape=(1,), dtype=float32)
tf.Tensor(0.8439249, shape=(), dtype=float32)
1
tf.Tensor([0.72720945], shape=(1,), dtype=float32)
tf.Tensor(0.72720945, shape=(), dtype=float32)
3
tf.Tensor([0.9678083  0.65831405 0.34201184], shape=(3,), dtype=float32)
tf.Tensor(0.34201184, shape=(), dtype=float32)
3
tf.Tensor([0.76596296 0.41130498 0.3626885 ], shape=(3,), dtype=float32)
tf.Tensor(0.3626885, shape=(), dtype=float32)
3
tf.Tensor([0.96950746 0.933819   0.92188597], shape=(3,), dtype=float32)
tf.Tensor(0.92188597, shape=(), dtype=float32)
1
tf.Tensor([0.99261636], shape=(1,), dtype=float32)
tf.Tensor(0.99261636, shape=(), dtype=float32)
1
tf.Tensor([0.9928888], shape=(1,), dtype=float32)
tf.Tensor(0.9928888, shape=(), dtype=float32)
1
tf.Tensor([0.9916065], shape=(1,), dtype=float32)
tf.Tensor(0.9916065, shape=(), dtype=float32)
1
tf.Tensor([0.9960025], shape=(1,), dtype=float32)
tf.Tensor(0.9960025, shape=(), dtype=float32)
1
tf.Tensor([0.99309033], shape=(1,), dtype

3
tf.Tensor([0.939548   0.8897522  0.07278407], shape=(3,), dtype=float32)
tf.Tensor(0.07278407, shape=(), dtype=float32)
3
tf.Tensor([9.9332857e-01 1.6994964e-03 6.7582284e-04], shape=(3,), dtype=float32)
tf.Tensor(0.00067582284, shape=(), dtype=float32)
4
tf.Tensor([0.7569866  0.6564815  0.5979282  0.19401915], shape=(4,), dtype=float32)
tf.Tensor(0.19401915, shape=(), dtype=float32)
4
tf.Tensor([0.9910697  0.765905   0.29064673 0.18177861], shape=(4,), dtype=float32)
tf.Tensor(0.18177861, shape=(), dtype=float32)
4
tf.Tensor([0.85098356 0.8078402  0.6965893  0.3961792 ], shape=(4,), dtype=float32)
tf.Tensor(0.3961792, shape=(), dtype=float32)
1
tf.Tensor([0.98976237], shape=(1,), dtype=float32)
tf.Tensor(0.98976237, shape=(), dtype=float32)
1
tf.Tensor([0.9919403], shape=(1,), dtype=float32)
tf.Tensor(0.9919403, shape=(), dtype=float32)
1
tf.Tensor([0.99245995], shape=(1,), dtype=float32)
tf.Tensor(0.99245995, shape=(), dtype=float32)
1
tf.Tensor([0.77013487], shape=(1,), dtype=floa

1
tf.Tensor([0.892778], shape=(1,), dtype=float32)
tf.Tensor(0.892778, shape=(), dtype=float32)
1
tf.Tensor([0.8043227], shape=(1,), dtype=float32)
tf.Tensor(0.8043227, shape=(), dtype=float32)
2
tf.Tensor([0.95662934 0.0291521 ], shape=(2,), dtype=float32)
tf.Tensor(0.029152099, shape=(), dtype=float32)
2
tf.Tensor([0.8720739 0.7093959], shape=(2,), dtype=float32)
tf.Tensor(0.7093959, shape=(), dtype=float32)
2
tf.Tensor([0.9102108 0.349792 ], shape=(2,), dtype=float32)
tf.Tensor(0.349792, shape=(), dtype=float32)
1
tf.Tensor([0.91887504], shape=(1,), dtype=float32)
tf.Tensor(0.91887504, shape=(), dtype=float32)
1
tf.Tensor([0.971593], shape=(1,), dtype=float32)
tf.Tensor(0.971593, shape=(), dtype=float32)
1
tf.Tensor([0.99655414], shape=(1,), dtype=float32)
tf.Tensor(0.99655414, shape=(), dtype=float32)
1
tf.Tensor([0.8727815], shape=(1,), dtype=float32)
tf.Tensor(0.8727815, shape=(), dtype=float32)
1
tf.Tensor([0.8468173], shape=(1,), dtype=float32)
tf.Tensor(0.8468173, shape=(), dt

ValueError: invalid literal for int() with base 10: '11-50'

In [2]:
groundtruth_csv_path = '/home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/data/LILA/db_export_season_all_cleaned.csv'
label_map_json = '/home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/data/LILA/label_map.json'

label_map_df = pd.DataFrame.from_dict(dataprep_utils.get_label_map_from_json(label_map_json), orient='index').reset_index()
label_map_df.columns=['species', 'labels']

groundtruth_df_img = pgc.process_grondtruth_classification_data(groundtruth_csv_path, label_map_df)

In [5]:
groundtruth_df_img.head()

,filename,species,labels,groundtruth_counts
0,S1/B04/B04_R1/S1_B04_R1_PICT0001,human,1,1
1,S1/B04/B04_R1/S1_B04_R1_PICT0002,human,1,2
2,S1/B04/B04_R1/S1_B04_R1_PICT0022,human,1,1
3,S1/B04/B04_R1/S1_B04_R1_PICT0023,human,1,1
4,S1/B04/B04_R1/S1_B04_R1_PICT0024,human,1,1


In [3]:
# groundtruth_df_img_grouped = groundtruth_df_img.groupby(by=['filename'], as_index=False)['groundtruth_counts'].sum()
groundtruth_dict = groundtruth_df_img.to_dict(orient='index')
dict1 = {}
for k, v in groundtruth_dict.items():
    if v['filename'] not in dict1.keys():
        dict1[v['filename']] = v['groundtruth_counts']
    elif v['groundtruth_counts'] == '11-50' and dict1[v['filename']] != '51+':
        dict1[v['filename']] = '11-50'
    elif v['groundtruth_counts'] != '51+' and dict1[v['filename']] == '11-50':
        dict1[v['filename']] = '11-50'
    elif v['groundtruth_counts'] == '51+' or dict1[v['filename']] == '51+':
        dict1[v['filename']] = '51+'
    else:
        dict1[v['filename']] = int(dict1[v['filename']])+int(v['groundtruth_counts'])

In [4]:
dict1['S6/O13/O13_R2/S6_O13_R2_IMAG3387']

'11-50'

In [7]:
groundtruth_df_img[groundtruth_df_img['filename']=='S4/C10/C10_R1/S4_C10_R1_IMAG1671']

,filename,species,labels,groundtruth_counts
329059,S4/C10/C10_R1/S4_C10_R1_IMAG1671,zebra,5,1
1243389,S4/C10/C10_R1/S4_C10_R1_IMAG1671,wildebeest,17,8


In [8]:
tfrecord_path_list = ['/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_s01_s06-10000-20000.record'] 
output_csv = '/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/snapshot_serengeti_s01_s06-10000-20000.csv'